In [ ]:
import os
from os import path

import pandas as pd
import json

In [ ]:
# Various datasets to visualize

option = 13
time0 = time1 = ''

if option == 13:    
    title = "SPIKE TEST"
    base_folder = "./simulations/spike2"
    sim_id = 141
    pathogen_id = 16
    sim_tz = "US/Eastern"
    time0 = 'Jun 11 2023 10:10AM'
    time1 = 'Jun 11 2023 11:10AM'
    time_step_min = 1
    use_new_id_schema = True

In [ ]:
data_folder = path.join(base_folder, "data")
output_root = path.join(base_folder, "output")
output_folder = path.join(output_root, "contacts")

In [ ]:
all_users = pd.read_csv(path.join(data_folder, "participants.csv"))
all_sequences = pd.read_csv(path.join(data_folder, "sequences.csv"))
all_mutations = pd.read_csv(path.join(data_folder, "mutations.csv"))

users = all_users[all_users["sim_id"] == sim_id]
ref_seq = list(all_sequences[all_sequences["pathogen_id"] == pathogen_id]["sequence"][0])
mutations = all_mutations[all_mutations["sim_id"] == sim_id]
mutations = mutations.assign(sequence='')

In [ ]:
print(''.join(ref_seq))

In [ ]:
mutations

In [ ]:
for idx in mutations.index:
    p_mut = mutations["prev_mutation_id"][idx]
    if p_mut == 0:
        seq0 = ref_seq
    else:
        seq0 = list(mutations[mutations["id"] == p_mut]["sequence"].values[0])
    delta = json.loads(mutations["delta"][idx])
    for k in delta.keys():
        pos = int(k)
        nt0, nt1 = delta[k].split('-')
        seq0[pos] = nt1
    mutations.at[idx, "sequence"] = seq0
    print(">seq" +  str(mutations["id"][idx]))
    print(''.join(seq0))
    


### Generate alignemnt (and trees) with https://www.ebi.ac.uk/Tools/msa/clustalo/